In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Date, Time, Boolean
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy.engine.reflection import Inspector
from pydantic import BaseModel, Field, validator, field_validator, ValidationError
import datetime
import pandas as pd
from typing import Optional

Base = declarative_base()


# Define the SQLAlchemy ORM model
class PlannerModel(Base):
    __tablename__ = 'planner'
    planner_id = Column(Integer, primary_key=True)
    planner_id_excel = Column(String, unique=True)
    ttver = Column(Integer)
    staff_code = Column(String)
    period_date = Column(Date)
    period_of_day = Column(Integer)
    period_code = Column(String)
    week_type = Column(String)
    period_is_lesson = Column(Integer)
    period_is_type = Column(String)
    period_mod = Column(String)
    period_notes = Column(String)
    period_json = Column(String)
    period_start_time = Column(Time)
    period_finish_time = Column(Time)
    room = Column(String)
    class_key_stage = Column(Integer)
    class_year = Column(Integer)
    class_code = Column(String)
    class_mod = Column(String)
    class_notes = Column(String)
    class_json = Column(String)
    lesson_sub_enter = Column(String)
    lesson_topic_enter = Column(String)
    lesson_lesson_enter = Column(String)
    lesson_topic_code = Column(String)
    lesson_code = Column(String)
    lesson_mod = Column(String)
    lesson_notes = Column(String)
    lesson_json = Column(String)

# Define the Pydantic Schema
class PlannerSchema(BaseModel):
    planner_id: int = Field(alias='PlannerID')
    planner_id_excel: Optional[str] = Field(alias='PlannerIDExcel', default=None)
    ttver: int = Field(alias='TTVer')
    staff_code: str = Field(alias='StaffCode')
    period_date: datetime.date = Field(alias='PeriodDate')
    period_of_day: int = Field(alias='PeriodOfDay')
    period_code: str = Field(alias='PeriodCode')
    week_type: str = Field(alias='WeekType')
    period_is_lesson: Optional[int] = Field(alias='PeriodIsLesson', default=None)
    period_is_type: Optional[str] = Field(alias='PeriodIsType', default=None)
    period_mod: Optional[str] = Field(alias='PeriodMod', default=None)
    period_notes: Optional[str] = Field(alias='PeriodNotes', default=None)
    period_json: Optional[str] = Field(alias='PeriodJSON', default=None)
    period_start_time: datetime.time = Field(alias='PeriodStartTime')
    period_finish_time: datetime.time = Field(alias='PeriodFinishTime')
    room: Optional[str] = Field(alias='Room', defualt=None)
    class_key_stage: Optional[int] = Field(alias='ClassKeyStage', default=None)
    class_year: Optional[int] = Field(alias='ClassYear', default=None)
    class_code: Optional[str] = Field(alias='ClassCode', default=None)
    class_mod: Optional[str] = Field(alias='ClassMod', default=None)
    class_notes: Optional[str] = Field(alias='ClassNotes', default=None)
    class_json: Optional[str] = Field(alias='ClassJSON', default=None)
    lesson_sub_enter: Optional[str] = Field(alias='SubEnter', default=None)
    lesson_topic_enter: Optional[str] = Field(alias='TopicEnter', default=None)
    lesson_lesson_enter: Optional[str] = Field(alias='LessonEnter', default=None)
    lesson_topic_code: Optional[str] = Field(alias='TopicCode', default=None)
    lesson_code: Optional[str] = Field(alias='LessonCode', default=None)
    lesson_mod: Optional[str] = Field(alias='LessonMod', default=None)
    lesson_notes: Optional[str] = Field(alias='LessonNotes', default=None)
    lesson_json: Optional[str] = Field(alias='LessonJSON', default=None)

    class Config:
        orm_mode = True
        populate_by_name = True  # to allow field aliases
    
    # Define the validators using the new Pydantic syntax
    @validator('planner_id', 'ttver', 'period_of_day', 'class_key_stage', 'class_year', pre=True)
    def parse_int(cls, v):
        return int(v) if v is not None else None

    @validator('planner_id_excel', 'staff_code', 'period_code', 'week_type', 'period_is_type', 'period_mod', 'period_notes',
               'period_json', 'room', 'class_code', 'class_mod', 'class_notes', 'class_json',
               'lesson_sub_enter', 'lesson_topic_enter', 'lesson_lesson_enter', 'lesson_topic_code', 'lesson_code',
               'lesson_mod', 'lesson_notes', 'lesson_json', pre=True)
    def parse_str(cls, v):
        return str(v) if v is not None else None

    @validator('period_date', 'period_start_time', 'period_finish_time', pre=True)
    def parse_datetime(cls, v):
        if v is not None:
            if isinstance(v, datetime.date) or isinstance(v, datetime.time):
                return v
            elif isinstance(v, str):
                try:
                    return pd.to_datetime(v).date() if '-' in v else datetime.datetime.strptime(v, '%H:%M:%S').time()
                except ValueError:
                    return None
        return None

# Function to convert from Pydantic Schema to SQLAlchemy Model
def schema_to_orm(schema: PlannerSchema) -> PlannerModel:
    return PlannerModel(
        planner_id=schema.planner_id,
        planner_id_excel=schema.planner_id_excel,
        ttver=schema.ttver,
        staff_code=schema.staff_code,
        period_date=schema.period_date,
        period_of_day=schema.period_of_day,
        period_code=schema.period_code,
        week_type=schema.week_type,
        period_is_lesson=schema.period_is_lesson,
        period_is_type=schema.period_is_type,
        period_mod=schema.period_mod,
        period_notes=schema.period_notes,
        period_json=schema.period_json,
        period_start_time=schema.period_start_time,
        period_finish_time=schema.period_finish_time,
        room=schema.room,
        class_key_stage=schema.class_key_stage,
        class_year=schema.class_year,
        class_code=schema.class_code,
        class_mod=schema.class_mod,
        class_notes=schema.class_notes,
        class_json=schema.class_json,
        lesson_sub_enter=schema.lesson_sub_enter,
        lesson_topic_enter=schema.lesson_topic_enter,
        lesson_lesson_enter=schema.lesson_lesson_enter,
        lesson_code=schema.lesson_code,
        lesson_mod=schema.lesson_mod,
        lesson_notes=schema.lesson_notes,
        lesson_json=schema.lesson_json
    )

class StaffModel(BaseModel):
    staff_id: int = Field(alias='StaffID')
    staff_type: str = Field(alias='StaffType')
    staff_code: str = Field(alias='StaffCode')
    staff_name: str = Field(alias='StaffName')
    staff_nickname: str = Field(alias='StaffNickname')
    staff_email: str = Field(alias='StaffEmail')
    staff_onenote: str = Field(alias='StaffOneNote')
    staff_json: str = Field(alias='StaffJSON')

    class Config:
        populate_by_name = True

    # Define the validators using the new Pydantic syntax

    @field_validator('staff_id')
    def _parse_staff_id(cls, v):
        return int(v) if pd.notnull(v) else None
    
    @field_validator('staff_type')
    def _parse_staff_type(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('staff_code')
    def _parse_staff_code(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('staff_name')
    def _parse_staff_name(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('staff_nickname')
    def _parse_staff_nickname(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('staff_email')
    def _parse_staff_email(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('staff_onenote')
    def _parse_staff_onenote(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('staff_json')
    def _parse_staff_json(cls, v):
        return str(v) if pd.notnull(v) else None

class StudentModel(BaseModel):
    student_id: int = Field(alias='StudentID')
    student_code: str = Field(alias='StudentCode')
    student_year: str = Field(alias='StudentYear')
    student_name: str = Field(alias='StudentName')
    student_nickname: str = Field(alias='StudentNickname')
    student_email: str = Field(alias='StudentEmail')

    class Config:
        populate_by_name = True

    # Define the validators using the new Pydantic syntax

    @field_validator('student_id')
    def _parse_student_id(cls, v):
        return int(v) if pd.notnull(v) else None
    
    @field_validator('student_code')
    def _parse_student_code(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('student_year')
    def _parse_student_year(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('student_name')
    def _parse_student_name(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('student_nickname')
    def _parse_student_nickname(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('student_email')
    def _parse_student_email(cls, v):
        return str(v) if pd.notnull(v) else None
    
class ClassModel(BaseModel):
    class_id: int = Field(alias='ClassID')
    class_code: str = Field(alias='ClassCode')
    class_name: str = Field(alias='ClassName')
    class_type: str = Field(alias='ClassType')
    class_year: str = Field(alias='ClassYear')
    class_students: str = Field(alias='ClassStudents')
    class_json: str = Field(alias='ClassJSON')

    class Config:
        populate_by_name = True

    # Define the validators using the new Pydantic syntax

    @field_validator('class_id')
    def _parse_class_id(cls, v):
        return int(v) if pd.notnull(v) else None
    
    @field_validator('class_code')
    def _parse_class_code(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('class_name')
    def _parse_class_name(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('class_type')
    def _parse_class_type(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('class_year')
    def _parse_class_year(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('class_students')
    def _parse_class_students(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('class_json')
    def _parse_class_json(cls, v):
        return str(v) if pd.notnull(v) else None

class ClassTimeTableModel(BaseModel):
    timetable_id: int = Field(alias='TimetableID')
    ttver: int = Field(alias='TTVer')
    class_code: str = Field(alias='ClassCode')
    week_type: str = Field(alias='WeekType')
    day_of_week: int = Field(alias='DayOfWeek')
    period_of_day: int = Field(alias='PeriodOfDay')
    room: str = Field(alias='Room')
    staff_code: str = Field(alias='StaffCode')
    period_code: str = Field(alias='PeriodCode')

    class Config:
        populate_by_name = True

    # Define the validators using the new Pydantic syntax

    @field_validator('timetable_id')
    def _parse_timetable_id(cls, v):
        return int(v) if pd.notnull(v) else None
    
    @field_validator('ttver')
    def _parse_ttver(cls, v):
        return int(v) if pd.notnull(v) else None
    
    @field_validator('class_code')
    def _parse_class_code(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('week_type')
    def _parse_week_type(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('day_of_week')
    def _parse_day_of_week(cls, v):
        return int(v) if pd.notnull(v) else None
    
    @field_validator('period_of_day')
    def _parse_period_of_day(cls, v):
        return int(v) if pd.notnull(v) else None
    
    @field_validator('room')
    def _parse_room(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('staff_code')
    def _parse_staff_code(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('period_code')
    def _parse_period_code(cls, v):
        return str(v) if pd.notnull(v) else None

class StaffTimetableModel(BaseModel):
    timetable_id: int = Field(alias='TimetableID')
    ttver: int = Field(alias='TTVer')
    staff_code: str = Field(alias='StaffCode')
    week_type: str = Field(alias='WeekType')
    day_of_week: int = Field(alias='DayOfWeek')
    period_of_day: int = Field(alias='PeriodOfDay')
    room: str = Field(alias='Room')
    class_code: str = Field(alias='ClassCode')
    period_code: str = Field(alias='PeriodCode')

    class Config:
        populate_by_name = True

    # Define the validators using the new Pydantic syntax

    @field_validator('timetable_id')
    def _parse_timetable_id(cls, v):
        return int(v) if pd.notnull(v) else None
    
    @field_validator('ttver')
    def _parse_ttver(cls, v):
        return int(v) if pd.notnull(v) else None
    
    @field_validator('staff_code')
    def _parse_staff_code(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('week_type')
    def _parse_week_type(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('day_of_week')
    def _parse_day_of_week(cls, v):
        return int(v) if pd.notnull(v) else None
    
    @field_validator('period_of_day')
    def _parse_period_of_day(cls, v):
        return int(v) if pd.notnull(v) else None
    
    @field_validator('room')
    def _parse_room(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('class_code')
    def _parse_class_code(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('period_code')
    def _parse_period_code(cls, v):
        return str(v) if pd.notnull(v) else None

class StudentTimetableModel(BaseModel):
    timetable_id: int = Field(alias='TimetableID')
    ttver: int = Field(alias='TTVer')
    student_code: str = Field(alias='StudentCode')
    week_type: str = Field(alias='WeekType')
    day_of_week: int = Field(alias='DayOfWeek')
    period_of_day: int = Field(alias='PeriodOfDay')
    room: str = Field(alias='Room')
    class_code: str = Field(alias='ClassCode')
    period_code: str = Field(alias='PeriodCode')

    class Config:
        populate_by_name = True

    # Define the validators using the new Pydantic syntax

    @field_validator('timetable_id')
    def _parse_timetable_id(cls, v):
        return int(v) if pd.notnull(v) else None
    
    @field_validator('ttver')
    def _parse_ttver(cls, v):
        return int(v) if pd.notnull(v) else None
    
    @field_validator('student_code')
    def _parse_student_code(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('week_type')
    def _parse_week_type(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('day_of_week')
    def _parse_day_of_week(cls, v):
        return int(v) if pd.notnull(v) else None
    
    @field_validator('period_of_day')
    def _parse_period_of_day(cls, v):
        return int(v) if pd.notnull(v) else None
    
    @field_validator('room')
    def _parse_room(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('class_code')
    def _parse_class_code(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('period_code')
    def _parse_period_code(cls, v):
        return str(v) if pd.notnull(v) else None
    
class PeriodModel(BaseModel):

    period_code: str = Field(alias='PeriodCode')
    period_name: str = Field(alias='PeriodName')
    period_week_type: str = Field(alias='PeriodWeekType')
    period_start_time: datetime.time = Field(alias='PeriodStartTime')
    period_finish_time: datetime.time = Field(alias='PeriodFinishTime')
    period_is_lesson: int = Field(alias='PeriodIsLesson')
    period_is_type: str = Field(alias='PeriodIsType')
    period_mod: str = Field(alias='PeriodMod')
    period_notes: str = Field(alias='PeriodNotes')
    period_json: str = Field(alias='PeriodJSON')

    class Config:
        populate_by_name = True

    # Define the validators using the new Pydantic syntax

    @field_validator('period_code')
    def _parse_period_code(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('period_name')
    def _parse_period_name(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('period_week_type')
    def _parse_period_week_type(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('period_start_time')
    def _parse_period_start_time(cls, v):
        return pd.to_datetime(v).time() if pd.notnull(v) else None
    
    @field_validator('period_finish_time')
    def _parse_period_finish_time(cls, v):
        return pd.to_datetime(v).time() if pd.notnull(v) else None
    
    @field_validator('period_is_lesson')
    def _parse_period_is_lesson(cls, v):
        return int(v) if pd.notnull(v) else None
    
    @field_validator('period_is_type')
    def _parse_period_is_type(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('period_mod')
    def _parse_period_mod(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('period_notes')
    def _parse_period_notes(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('period_json')
    def _parse_period_json(cls, v):
        return str(v) if pd.notnull(v) else None
    
class LessonTypeModel(BaseModel):
    
    lesson_type_code: str = Field(alias='LessonTypeCode')
    lesson_type: str = Field(alias='LessonTypeName')
    lesson_type_notes: str = Field(alias='LessonTypeNotes')
    lesson_type_json: str = Field(alias='LessonTypeJSON')

    class Config:
        populate_by_name = True

    # Define the validators using the new Pydantic syntax

    @field_validator('lesson_type_code')
    def _parse_lesson_code(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('lesson_type')
    def _parse_lesson_name(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('lesson_type_notes')
    def _parse_lesson_notes(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('lesson_type_json')
    def _parse_lesson_json(cls, v):
        return str(v) if pd.notnull(v) else None
        
class LessonModel(BaseModel):
        
    lesson_code: str = Field(alias='LessonCode')
    lesson_name: str = Field(alias='LessonName')
    lesson_mod: str = Field(alias='LessonMod')
    lesson_notes: str = Field(alias='LessonNotes')
    lesson_json: str = Field(alias='LessonJSON')

    class Config:
        populate_by_name = True

    # Define the validators using the new Pydantic syntax

    @field_validator('lesson_code')
    def _parse_lesson_code(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('lesson_name')
    def _parse_lesson_name(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('lesson_mod')
    def _parse_lesson_mod(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('lesson_notes')
    def _parse_lesson_notes(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('lesson_json')
    def _parse_lesson_json(cls, v):
        return str(v) if pd.notnull(v) else None
            
class RoomModel(BaseModel):
            
    room_code: str = Field(alias='RoomCode')
    room_name: str = Field(alias='RoomName')
    room_notes: str = Field(alias='RoomNotes')
    room_json: str = Field(alias='RoomJSON')

    class Config:
        populate_by_name = True

    # Define the validators using the new Pydantic syntax

    @field_validator('room_code')
    def _parse_room_code(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('room_name')
    def _parse_room_name(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('room_notes')
    def _parse_room_notes(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('room_json')
    def _parse_room_json(cls, v):
        return str(v) if pd.notnull(v) else None
                
class LessonPeriodModel(BaseModel):
                
    lesson_period_code: str = Field(alias='LessonPeriodCode')
    lesson_period_name: str = Field(alias='LessonPeriodName')
    lesson_period_notes: str = Field(alias='LessonPeriodNotes')
    lesson_period_json: str = Field(alias='LessonPeriodJSON')

    class Config:
        populate_by_name = True

    # Define the validators using the new Pydantic syntax

    @field_validator('lesson_period_code')
    def _parse_lesson_period_code(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('lesson_period_name')
    def _parse_lesson_period_name(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('lesson_period_notes')
    def _parse_lesson_period_notes(cls, v):
        return str(v) if pd.notnull(v) else None
    
    @field_validator('lesson_period_json')
    def _parse_lesson_period_json(cls, v):
        return str(v) if pd.notnull(v) else None


In [ ]:
# If the file exists, delete it
import os
if os.path.exists('tmp/kevlar_ai.db'):
    os.remove('tmp/kevlar_ai.db')

# SQLAlchemy engine and session creation to create the tables in the database
engine = create_engine('sqlite:///tmp/kevlar_ai.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)

# Read the Excel file
file_path = 'planner.xlsx'  # Replace with your file path
planner_df = pd.read_excel(file_path, sheet_name='Planner')

# Insert the data into the database
session = Session()
i = 0
for _, row in planner_df.iterrows():
    # If row['PeriodMod'] == 'H', skip the row
    if row['PeriodMod'] == 'H':
        continue
    try:
        planner_schema = PlannerSchema(
            planner_id=i,
            planner_id_excel=str(row['StaffWeekPeriodCode']) if pd.notnull(row['StaffWeekPeriodCode']) else None,
            ttver=row['TTVer'],
            staff_code=row['StaffCode'],
            week_type=row['WeekAOrBOfTimetable'],
            period_of_day=row['PeriodOfDay'],
            period_code=row['PeriodCode'],
            period_date=pd.to_datetime(row['LessonDate']) if pd.notnull(row['LessonDate']) else None,
            period_mod=str(row['PeriodMod']) if pd.notnull(row['PeriodMod']) else None,
            period_start_time=convert_to_HERE IS MY CURRENT CODEtime(row.get('LessonStartTime')),
            period_finish_time=convert_to_time(row.get('LessonFinishTime')),
            period_is_lesson=None,
            period_is_type=None,
            period_notes=None,
            period_json=None,
            room=str(row['Room']) if pd.notnull(row['Room']) else None,
            class_key_stage=row['KeyStage'] if pd.notnull(row['KeyStage']) else None,
            class_year=row['YearGroup'] if pd.notnull(row['YearGroup']) else None,
            class_code=str(row['ClassCode']) if pd.notnull(row['ClassCode']) else None,
            class_mod=None,
            class_notes=None,
            class_json=None,
            lesson_sub_enter=str(row['SubEnter']) if pd.notnull(row['SubEnter']) else None,
            lesson_topic_enter=str(row['TopicEnter']) if pd.notnull(row['TopicEnter']) else None,
            lesson_lesson_enter=str(row['LessonEnter']) if pd.notnull(row['LessonEnter']) else None,
            lesson_topic_code=str(row['TopicCode']) if pd.notnull(row['TopicCode']) else None,
            lesson_code=str(row['TopicLessonCode']) if pd.notnull(row['TopicLessonCode']) else None,
            lesson_mod=None,
            lesson_notes=str(row['LessonNotesEnter']) if pd.notnull(row['LessonNotesEnter']) else None,
            lesson_json=str(row['LessonJSON']) if pd.notnull(row['LessonJSON']) else None,
        )
        # Convert schema to ORM
        planner_orm = schema_to_orm(planner_schema)
        session.add(planner_orm)
        i += 1
    except ValidationError as exc:
        print(repr(exc.errors()[0]))

# Commit the transaction
session.commit()

print("Data from the Planner sheet has been inserted into the database.")

In [ ]:
# Print some details about the database (it only contains one table for now)
inspector = Inspector.from_engine(engine)
print("Number of rows in the first table: ", inspector.get_table_names()[0], session.query(PlannerModel).count())
print("Columns in the first table: ", inspector.get_columns(inspector.get_table_names()[0]))
print("First row in the first table: ", session.query(PlannerModel).first().planner_id_excel)
print("Last row in the first table: ", session.query(PlannerModel).order_by(PlannerModel.planner_id_excel.desc()).first().planner_id_excel)

In [ ]:
# Function to get a list of all the staff members
def get_staff():
    results = session.query(PlannerModel).all()
    staff = []
    for row in results:
        if row.staff_code not in staff:
            staff.append(row.staff_code)
    return staff

# Function to get a list of all the classes that a staff member teaches
def get_classes_for_staff(staff_code):
    results = session.query(PlannerModel).filter(PlannerModel.staff_code == staff_code).all()
    classes = []
    for row in results:
        if row.class_code not in classes:
            classes.append(row.class_code)
    return classes

# Function to get a list of all the staff members that teach a class
def get_staff_for_class(class_code):
    results = session.query(PlannerModel).filter(PlannerModel.class_code == class_code).all()
    staff = []
    for row in results:
        if row.staff_code not in staff:
            staff.append(row.staff_code)
    return staff

# Function to filter and sort data
def filter_and_sort_data(session, staff_code=None, date_range=None, class_code=None, sort_by=None):
    query = session.query(PlannerModel)
    if staff_code:
        query = query.filter(PlannerModel.staff_code == staff_code)
    if date_range:
        query = query.filter(PlannerModel.period_date.between(*date_range))
    if class_code:
        query = query.filter(PlannerModel.class_code == class_code)
    if sort_by:
        query = query.order_by(sort_by)
    return query.all()

In [ ]:
# Create document for each class of each staff member and save it to a folder
import os
from docx import Document
from docx.shared import Inches

# Create a folder to store the documents
if not os.path.exists('tmp/staff_classes'):
    os.makedirs('tmp/staff_classes')

# Create a document for each class of each staff member
staff_list = get_staff()

for staff in staff_list:
    classes = get_classes_for_staff(staff)
    for class_code in classes:
        document = Document()
        document.add_heading(f"{staff} - {class_code}", 0)
        document.add_paragraph(f"Staff: {staff}")
        document.add_paragraph(f"Class: {class_code}")
        document.add_paragraph(f"Date: {datetime.datetime.now().strftime('%d %B %Y')}")
        document.add_paragraph(f"Time: {datetime.datetime.now().strftime('%H:%M:%S')}")
        document.add_paragraph(f"")
    
        # Get the data for the staff member and class
        results = filter_and_sort_data(session, staff_code=staff, class_code=class_code, sort_by=PlannerModel.period_date)
        for row in results:
            document.add_paragraph(f"Date: {row.period_date.strftime('%d %B %Y')}")
            document.add_paragraph(f"Start Time: {row.period_start_time.strftime('%H:%M:%S')}")
            document.add_paragraph(f"Finish Time: {row.period_finish_time.strftime('%H:%M:%S')}")
            document.add_paragraph(f"Room: {row.room}")
            document.add_paragraph(f"Lesson Code: {row.lesson_code}")
            document.add_paragraph(f"Lesson Notes: {row.lesson_notes}")
            document.add_paragraph(f"")
        # Save the file, replacing the "/" character in the class code with "-"
        document.save(f"tmp/staff_classes/{staff}_{class_code.replace('/', '-')}.docx")

In [ ]:
# Function to update data
def update_data(session, planner_id, update_fields):
    query = session.query(PlannerModel).filter(PlannerModel.planner_id == planner_id)
    query.update(update_fields)
    session.commit()

In [ ]:
# Function to shift lesson data
def shift_lesson_data(session, planner_id, shift_amount):
    # Implement logic to shift the lesson data
    # This involves updating multiple rows and ensuring no conflicts in the schedule
    # planner_id=schema.planner_id,
    # planner_id_excel=schema.planner_id_excel,
    # ttver=schema.ttver,
    # staff_code=schema.staff_code,
    # period_date=schema.period_date,
    # period_of_day=schema.period_of_day,
    # period_code=schema.period_code,
    # week_type=schema.week_type,
    # period_is_lesson=schema.period_is_lesson,
    # period_is_type=schema.period_is_type,
    # period_mod=schema.period_mod,
    # period_notes=schema.period_notes,
    # period_json=schema.period_json,
    # period_start_time=schema.period_start_time,
    # period_finish_time=schema.period_finish_time,
    # room=schema.room,
    # class_key_stage=schema.class_key_stage,
    # class_year=schema.class_year,
    # class_code=schema.class_code,
    # class_mod=schema.class_mod,
    # class_notes=schema.class_notes,
    # class_json=schema.class_json,
    # lesson_sub_enter=schema.lesson_sub_enter,
    # lesson_topic_enter=schema.lesson_topic_enter,
    # lesson_lesson_enter=schema.lesson_lesson_enter,
    # lesson_code=schema.lesson_code,
    # lesson_mod=schema.lesson_mod,
    # lesson_notes=schema.lesson_notes,
    # lesson_json=schema.lesson_json

    pass


In [ ]:
# Example usage
with Session() as session:
    # Filter and sort
    lessons = filter_and_sort_data(session, staff_code='KCA', sort_by=PlannerModel.period_date)
    
    # Update a specific lesson
    update_data(session, planner_id=123, update_fields={'room': 'New Room', 'period_start_time': new_start_time})

    # Shift lesson data
    shift_lesson_data(session, planner_id=123, shift_amount=1)  # Example: shift by 1 period